In [1]:
%reload_ext autoreload
%autoreload 2

import optuna

c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


##### Hyperparameter Tuning with Accuracy Pruning


In [3]:
import models_hyper_optim_optuna
import models
import neural_net_model_train


#set folders where models will be saved
def setFolders(evaluation_function, use_gru_instead_of_lstm):    
    if use_gru_instead_of_lstm:
        gru_path_root_model = "models\cnn\gru\\" + evaluation_function
        models_hyper_optim_optuna.createFolder(gru_path_root_model) 
        
        return gru_path_root_model +  "\cnn_bi_gru_"
    else:
        lstm_path_root_model = "models\cnn\lstm\\" + evaluation_function
        models_hyper_optim_optuna.createFolder(lstm_path_root_model) 
        
        return lstm_path_root_model + "\cnn_bi_lstm_"
    

c:\Users\androgo\Documents\Python Scripts\IDS\CICIDS2017_notebooks\IDS_Project
X_train__23_features (1500, 23)
X_test__23_features (1500, 23)
y_train_binary__23_features (1500,)
0.0    1500
dtype: int64
Y_test__binary (1500,)
classes_y:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
class_weights:  [1.17647059 0.90909091 0.89285714 0.96153846 1.         0.8
 0.94339623 0.97087379 1.0989011  1.04166667 1.13636364 0.95238095
 1.01010101 1.31578947 1.        ]

classes_class_weights: 
{0: 1.176,
 1: 0.909,
 2: 0.893,
 3: 0.962,
 4: 1.0,
 5: 0.8,
 6: 0.943,
 7: 0.971,
 8: 1.099,
 9: 1.042,
 10: 1.136,
 11: 0.952,
 12: 1.01,
 13: 1.316,
 14: 1.0}


c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


DEVICE found:  cuda:0
number of classes:  15


In [4]:
channels = 1
batch_size= 32

args_model = {
    'number_of_classes':models_hyper_optim_optuna.number_of_classes, 
    'input_shape' : (batch_size, channels, models_hyper_optim_optuna.input_size),
    'use_gru_instead_of_lstm':False
    }

args_trial = {
    'max_epochs' : 7,
    'no_trials': 7, 
    'evaluation_function':'accuracy', 
    'USE_AUTOMATIC_MIXED_PRECISION':False
}



#set models that will be used for tunning
cnn_birnn_model = models.CnnBirnnModel(args_model)
models_trainning = neural_net_model_train.ModelsTrainning()

### Accuracy Tunning

In [5]:

from time import time


args_trial['evaluation_function']='accuracy'
path_models_id = setFolders(args_trial['evaluation_function'], args_model['use_gru_instead_of_lstm'])

start = time()


study = optuna.create_study(direction = "maximize", sampler = optuna.samplers.TPESampler(seed=1121218),
                            pruner=optuna.pruners.HyperbandPruner(min_resource=3))

models_hyper_optim_optuna.start_optimize_objective(study, args_trial,
                                                   path_models_id,
                                                   cnn_birnn_model,
                                                   models_trainning)

end = time()


[I 2022-09-28 11:53:45,525] A new study created in memory with name: no-name-0ab985fc-6eca-410c-b875-8f018fa22f8a
c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


NetModel(
  (cv1): Conv1d(1, 115, kernel_size=(3,), stride=(3,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
  (bn1): BatchNorm1d(115, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(115, 151, kernel_size=(1,), stride=(3,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn2): BatchNorm1d(151, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(151, 139, kernel_size=(1,), stride=(3,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn3): BatchNorm1d(139, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(139, 208, kernel_size=(1,), stride=(7,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(7,), padding=(0,))
  (bn4): BatchNorm1d(208, eps=1e-05, momentum=0.1, affin

100%|██████████| 46/46 [00:01<00:00, 25.87it/s]



Train set: Average loss: 0.0664, Average accuracy: 3.1460%,
 	 Average f1_score: 0.0610, Average Area Under ROC: 0.4987 



100%|██████████| 46/46 [00:00<00:00, 59.20it/s]
[I 2022-09-28 11:54:05,595] Trial 0 finished with value: 0.07201086729764938 and parameters: {'n_layers': 7, 'n_units_l0': 115, 'no_strides_l0': 3, 'n_units_l1': 151, 'no_strides_l1': 3, 'n_units_l2': 139, 'no_strides_l2': 3, 'n_units_l3': 208, 'no_strides_l3': 7, 'n_units_l4': 34, 'no_strides_l4': 5, 'n_units_l5': 217, 'no_strides_l5': 7, 'n_units_l6': 58, 'no_strides_l6': 1, 'dropout_l6': 0.17505252828469767, 'learning_rate': 0.09664150604604756, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.07201086729764938.


...

100%|██████████| 46/46 [00:01<00:00, 45.73it/s]
[I 2022-09-28 11:54:55,456] Trial 4 finished with value: 0.06317935138940811 and parameters: {'n_layers': 8, 'n_units_l0': 232, 'no_strides_l0': 5, 'n_units_l1': 154, 'no_strides_l1': 3, 'n_units_l2': 64, 'no_strides_l2': 3, 'n_units_l3': 97, 'no_strides_l3': 7, 'n_units_l4': 46, 'no_strides_l4': 3, 'n_units_l5': 187, 'no_strides_l5': 3, 'n_units_l6': 244, 'no_strides_l6': 1, 'n_units_l7': 166, 'no_strides_l7': 7, 'dropout_l7': 0.2360531735539034, 'learning_rate': 0.010737792310750963, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.07472825795412064.



Test set: Average loss: 2.9123, Average accuracy: 0.0632%,
 	  Average f1_score: 0.0571, Average Area Under ROC: 0.4939 

NetModel(
  (cv1): Conv1d(1, 133, kernel_size=(3,), stride=(3,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
  (bn1): BatchNorm1d(133, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(133, 55, kernel_size=(1,), stride=(5,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(5,), padding=(0,))
  (bn2): BatchNorm1d(55, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(55, 184, kernel_size=(1,), stride=(5,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(5,), padding=(0,))
  (bn3): BatchNorm1d(184, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(184, 100, kernel_size=(1,), stride=(5,), bias=False)
  

100%|██████████| 46/46 [00:02<00:00, 16.21it/s]
[I 2022-09-28 11:55:07,789] Trial 5 pruned. 


NetModel(
  (cv1): Conv1d(1, 157, kernel_size=(3,), stride=(5,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(5,), padding=(0,))
  (bn1): BatchNorm1d(157, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(157, 70, kernel_size=(1,), stride=(1,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn2): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(70, 127, kernel_size=(1,), stride=(5,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(5,), padding=(0,))
  (bn3): BatchNorm1d(127, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(127, 97, kernel_size=(1,), stride=(1,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn4): BatchNorm1d(97, eps=1e-05, momentum=0.1, affine=Tru

100%|██████████| 46/46 [00:02<00:00, 16.42it/s]
[I 2022-09-28 11:55:20,414] Trial 6 pruned. 


In [6]:
trials_finished = study.get_trials()
print('number of trials finished: ', len(trials_finished),'\n')

print('number of the best trial: ',study.best_trial.number)

best_trial = study.best_trial
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

number of trials finished:  7 

number of the best trial:  1
n_layers: 8
n_units_l0: 172
no_strides_l0: 1
n_units_l1: 247
no_strides_l1: 3
n_units_l2: 241
no_strides_l2: 5
n_units_l3: 28
no_strides_l3: 3
n_units_l4: 88
no_strides_l4: 5
n_units_l5: 250
no_strides_l5: 5
n_units_l6: 241
no_strides_l6: 5
n_units_l7: 37
no_strides_l7: 7
dropout_l7: 0.23318976981174463
learning_rate: 1.2118050493295717e-05
optimizer: RMSprop


### Tuning Visualization

In [7]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [8]:
optuna.visualization.plot_optimization_history(study)

In [9]:
optuna.visualization.plot_parallel_coordinate(study)

[W 2022-09-28 11:55:22,565] Your study has only completed trials with missing parameters.


In [10]:
optuna.visualization.plot_param_importances(study)

In [11]:
optuna.visualization.plot_slice(study)


### F1 score Tunning

In [12]:

args_trial['evaluation_function']='f1_score'
path_models_id = setFolders(args_trial['evaluation_function'], args_model['use_gru_instead_of_lstm'])



study = optuna.create_study(direction = "maximize", sampler = optuna.samplers.TPESampler(seed=1121218),
                            pruner=optuna.pruners.HyperbandPruner(min_resource=3))

models_hyper_optim_optuna.start_optimize_objective(study, args_trial,
                                                   path_models_id,
                                                   cnn_birnn_model, 
                                                   models_trainning)


[I 2022-09-28 11:55:24,256] A new study created in memory with name: no-name-30583b3c-fde0-4b67-9d41-24abb21e0f9d
c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning:

Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.



NetModel(
  (cv1): Conv1d(1, 115, kernel_size=(3,), stride=(3,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
  (bn1): BatchNorm1d(115, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(115, 151, kernel_size=(1,), stride=(3,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn2): BatchNorm1d(151, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(151, 139, kernel_size=(1,), stride=(3,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn3): BatchNorm1d(139, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(139, 208, kernel_size=(1,), stride=(7,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(7,), padding=(0,))
  (bn4): BatchNorm1d(208, eps=1e-05, momentum=0.1, affin

100%|██████████| 46/46 [00:01<00:00, 25.93it/s]



Train set: Average loss: 0.0628, Average accuracy: 3.0089%,
 	 Average f1_score: 0.0576, Average Area Under ROC: 0.4993 



100%|██████████| 46/46 [00:00<00:00, 54.31it/s]
[I 2022-09-28 11:55:42,517] Trial 0 finished with value: 0.06126486510038376 and parameters: {'n_layers': 7, 'n_units_l0': 115, 'no_strides_l0': 3, 'n_units_l1': 151, 'no_strides_l1': 3, 'n_units_l2': 139, 'no_strides_l2': 3, 'n_units_l3': 208, 'no_strides_l3': 7, 'n_units_l4': 34, 'no_strides_l4': 5, 'n_units_l5': 217, 'no_strides_l5': 7, 'n_units_l6': 58, 'no_strides_l6': 1, 'dropout_l6': 0.17505252828469767, 'learning_rate': 0.09664150604604756, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.06126486510038376.


...

100%|██████████| 46/46 [00:02<00:00, 19.03it/s]



Train set: Average loss: 0.0674, Average accuracy: 2.7121%,
 	 Average f1_score: 0.0628, Average Area Under ROC: 0.5036 



100%|██████████| 46/46 [00:01<00:00, 38.18it/s]
[I 2022-09-28 11:56:37,135] Trial 5 finished with value: 0.06896299123764038 and parameters: {'n_layers': 9, 'n_units_l0': 133, 'no_strides_l0': 3, 'n_units_l1': 55, 'no_strides_l1': 5, 'n_units_l2': 184, 'no_strides_l2': 5, 'n_units_l3': 100, 'no_strides_l3': 5, 'n_units_l4': 88, 'no_strides_l4': 3, 'n_units_l5': 76, 'no_strides_l5': 3, 'n_units_l6': 37, 'no_strides_l6': 7, 'n_units_l7': 208, 'no_strides_l7': 1, 'n_units_l8': 211, 'no_strides_l8': 1, 'dropout_l8': 0.11161620992541219, 'learning_rate': 4.937184568545008e-05, 'optimizer': 'Adam'}. Best is trial 5 with value: 0.06896299123764038.



Test set: Average loss: 2.7093, Average accuracy: 0.0747%,
 	  Average f1_score: 0.0690, Average Area Under ROC: 0.5098 

NetModel(
  (cv1): Conv1d(1, 157, kernel_size=(3,), stride=(5,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(5,), padding=(0,))
  (bn1): BatchNorm1d(157, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(157, 70, kernel_size=(1,), stride=(1,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn2): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(70, 127, kernel_size=(1,), stride=(5,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(5,), padding=(0,))
  (bn3): BatchNorm1d(127, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(127, 97, kernel_size=(1,), stride=(1,), bias=False)
  (

100%|██████████| 46/46 [00:03<00:00, 15.31it/s]
[I 2022-09-28 11:56:49,322] Trial 6 pruned. 


In [13]:
trials_finished = study.get_trials()
print('number of trials finished: ', len(trials_finished),'\n')

print('number of the best trial: ',study.best_trial.number)

best_trial = study.best_trial
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

number of trials finished:  7 

number of the best trial:  5
n_layers: 9
n_units_l0: 133
no_strides_l0: 3
n_units_l1: 55
no_strides_l1: 5
n_units_l2: 184
no_strides_l2: 5
n_units_l3: 100
no_strides_l3: 5
n_units_l4: 88
no_strides_l4: 3
n_units_l5: 76
no_strides_l5: 3
n_units_l6: 37
no_strides_l6: 7
n_units_l7: 208
no_strides_l7: 1
n_units_l8: 211
no_strides_l8: 1
dropout_l8: 0.11161620992541219
learning_rate: 4.937184568545008e-05
optimizer: Adam


### Tuning Visualization

In [14]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [15]:
optuna.visualization.plot_optimization_history(study)

In [16]:
optuna.visualization.plot_parallel_coordinate(study)

[W 2022-09-28 11:56:50,512] Your study has only completed trials with missing parameters.


In [17]:
optuna.visualization.plot_param_importances(study)

In [18]:
optuna.visualization.plot_slice(study)